# Sea ice forecasting using the IceNet library

## Context
### Purpose
This notebook demonstrates the use of the [IceNet library](https://github.com/icenet-ai/icenet/) for sea-ice forecasting trained using climate reanalysis and observational data.

### Description
[IceNet](https://github.com/icenet-ai/icenet/) is a python library that provides the ability to download, process, train and predict from end to end. Users can interact with IceNet either via the python interface or via a set of command-line interfaces (CLI) which provide a high-level interface that covers the above abilities.

This notebook demonstrates the use of the python library api for forecasting sea ice for a reduced dataset to demonstrate its capabilities.

### Modelling approach
IceNet is a probabilistic, deep learning sea ice forecasting system. It utilises ensemble modelling of U-Net networks to generate daily forecasts of sea ice condition, trained on climate reanalysis and sea ice observational data at 25km resolution. The original IceNet research model, published in [Nature Communications](https://www.nature.com/articles/s41467-021-25257-4) was trained on climate simulations and observational data to forecast the next 6 months of monthly-averaged sea ice concentration maps. This version advanced the range of accurate sea ice forecasts, outperforming a state-of-the-art dynamical model (ECMWF SEAS5) in seasonal forecasts of summer sea ice, particularly for extreme sea ice events.

### Highlights
*Provide 3-5 bullet points that convey the use case’s core procedures. Each bullet point must have a maximum of 85 characters, including spaces.*

 * [1. Setup](#Setup): Set up the environment and project structure.
 * [2. Download](#Download): Download sea ice concentration data for training using the built-in downloader for first quarter of the year 2020.
 * [3. Process](#Process): Process the downloaded data by renormalising variables as needed, and generating cached datasets to speed up training.
 * [4. Train](#Train): Train the neural network and generate checkpoint best result.
 * [5. Predict](#Predict): Predict for defined dates.
 * [6. Visualisation](#Visualisation): Visualise the prediction output.

### Contributions

#### Notebook
* James Byrne (Notebook Author), British Antarctic Survey, [@JimCircadian](https://github.com/JimCircadian)
* Bryn Noel Ubald (Notebook Author), British Antarctic Survey, [@bnubald](https://github.com/bnubald)

#### Modelling codebase
* James Byrne (Code author)
* Tom Andersson (Science author)

__Please raise issues [in this repository](https://github.com/icenet-ai/icenet-notebooks/issues) to suggest updates to this notebook!__ 

Contact me at _jambyr \<at\> bas.ac.uk_ for anything else...

#### Modelling publications
Andersson, T.R., Hosking, J.S., Pérez-Ortiz, M. et al. Seasonal Arctic sea ice forecasting with probabilistic deep learning. Nat Commun 12, 5124 (2021). https://doi.org/10.1038/s41467-021-25257-4

> [!NOTE]  
> 
> IceNet has developed significantly since the inclusion of the existing notebook (relates to issue #6) based on the work in the original paper.
>
> The original paper and notebook used a combination of climate simulations and observational data to forecast the next 6 months of monthly-averaged sea ice concentration. Since then, the original code has been refactored into a new library icenet as showcased in this notebook.
> This library supports sea ice forecasting on a daily resolution rather than monthly-averaged. It has been developed significantly since the original paper, and there are multiple ways of interacting with the library to help enable development of sea ice forecasting and model development.

#### Involved organisations
The Alan Turing Institute and British Antarctic Survey

___
# Setup

## Load libraries
Load some of the common libraries required.

In [ ]:
import os
import random
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
import tensorflow as tf

# We also set the logging level so that we get some feedback from the API
import logging
logging.basicConfig(level=logging.INFO)

The following imports modules from the IceNet library as preparation for the downloaders. Whose instantiation describes the interactions with the upstream APIs/data interfaces used to source various types of data. 

In [ ]:
from icenet.data.sic.mask import Masks
from icenet.data.interfaces.cds import ERA5Downloader
from icenet.data.sic.osisaf import SICDownloader

## Set project structure
*The cell below creates a separate folder to save the notebook outputs. This facilitates the reader to inspect inputs/outputs stored within a defined destination folder. Don't remove the lines below.*

In [ ]:
notebook_folder = './notebook'
if not os.path.exists(notebook_folder):
    os.makedirs(notebook_folder)

___
# Download

In this section, we download all required data with our extended date range. All downloaders inherit a `download` method from the `Downloader` class in [`icenet.data.producers`](https://github.com/icenet-ai/icenet/blob/main/icenet/data/producers.py), which also contains two other data producing classes `Generator` (which Masks inherits from) and `Processor` (used in the next section), each providing abstract implementations that multiple classes derive from.

### Mask data

We start here with generating the masks for training/prediction. This includes regions where sea ice does not form, land regions, and the [polar hole](https://blogs.egu.eu/divisions/cr/2016/10/14/image-of-the-week-the-polar-hole/).

In [ ]:
masks = Masks(north=False, south=True)
masks.generate(save_polarhole_masks=False)

## Climate and Ocean data

Climate and ocean data are obtained from the [Climate Data Store (CDS)](https://cds.climate.copernicus.eu/).

The climate data used for training is from [ERA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview) reanalysis which covers the global climate since 1940 to the present time.

The Ocean data used is from [ORA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-oras5?tab=overview) which also uses a reanalysis approach and contains global ocean and sea-ice reanalysis data.

Since these are both obtained from reanalysis, they are a combination of physical models and observational data. Due to the reanalysis approach, there is no temporal or spatial gap in the downloaded data. Both of these sets of data are obtained from the ECMWF's (European Centre for Medium-Range Weather Forecast) reanalysis systems.

Please see the above links for more details on these datasets.

The downloader implementation of this data in IceNet utilises the CDS API which requires registration and configuration of a token before downloading. The registration is free, please see [here](https://cds.climate.copernicus.eu/api-how-to) for more instructions on how to set this up.

Assuming you have configured the CDS API key correctly, you will be able to download using the following IceNet class. Since the key is personal and should not be shared, the call to download an example dataset is shown below, but not used in this demonstrator notebook.

```python
era5 = ERA5Downloader(
    var_names=["tas", "zg", "uas", "vas"],      # Name of variables to download
    dates=[                                     # Dates to download the variable data for
        pd.to_datetime(date).date()
        for date in pd.date_range("2020-01-01", "2020-04-31", freq="D")
    ],
    path=data_dir,                              # Location to download data to (default is `./data`)
    delete_tempfiles=True,                      # Whether to delete temporary downloaded files
    levels=[None, [250, 500], None, None],      # The levels at which to obtain the variables for (e.g. for zg, it is the pressure levels)
    max_threads=4,                              # Maximum number of concurrent downloads
    north=False,                                # Boolean: Whether require data across northern hemisphere
    south=True,                                 # Boolean: Whether require data across southern hemisphere
    use_toolbox=False)                          # Experimental, alternative download method
era5.download()                                 # Start downloading
```

The `ERA5Downloader` inherits from `ClimateDownloader`, from which several implementations derive their functionality. Two particularly useful methods shown below allow the downloaded data to be converted to the same grid and orientation as the OSISAF sea-ice concentration (SIC) data in the next cell.

```python
era5.regrid()                                   # Map data onto common EASE2 grid
era5.rotate_wind_data()                         # Rotate wind data to correct orientation
```

## Sea-ice concentration (SIC) data

The sea-ice concentration data use for training is obtained from [OSI SAF](https://osi-saf.eumetsat.int/products/sea-ice-products).

The SIC is defined as the fraction of a grid cell that is covered in sea-ice.

You will notice a familiar interface as with the `ERA5Downloader` class with the `SICDownloader` class.

In [ ]:
sic = SICDownloader(
    dates=[
        pd.to_datetime(date).date()             # Dates to download the variable data for
        for date in pd.date_range("2020-01-01", "2020-03-31", freq="D")
    ],
    delete_tempfiles=True, # Whether to delete temporary downloaded files
    north=False,           # Boolean: Whether to use mask for this region
    south=True,            # Boolean: Whether to use mask for this region
    parallel_opens=True,   # Boolean: Whether to use `dask.delayed` to open and preprocess multiple files in parallel
)

sic.download()

___
# Process

Similarly to the downloaders, each data producer (be it a `Downloader` or `Generator`) has a respective `Processor` that converts the `./notebook/data/` products into a normalised, preprocessed dataset under `./notebook/processed/`.

Firstly, to make life a bit easier, we set up some variables. In this case we're creating a train/validate/test split out of the 2020 data in a fairly naive manner.

In [ ]:
processing_dates = dict(
    train=[pd.to_datetime(el) for el in pd.date_range("2020-01-01", "2020-03-06")],
    val=[pd.to_datetime(el) for el in pd.date_range("2020-03-11", "2020-03-31")],
    test=[pd.to_datetime(el) for el in pd.date_range("2020-03-08", "2020-03-09")],
)
processed_name = "notebook_api_data"

Next, we create the data producer and configure them for the dataset we want to create.

These modules import the Processing modules for the downloaded data.

In [ ]:
from icenet.data.processors.era5 import IceNetERA5PreProcessor
from icenet.data.processors.meta import IceNetMetaPreProcessor
from icenet.data.processors.osi import IceNetOSIPreProcessor

In [ ]:
osi = IceNetOSIPreProcessor(
    ["siconca"],                # Absolute normalised variables
    [],                         # Variables defined as deviations from an aggregated norm
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True,
)

meta = IceNetMetaPreProcessor(
    processed_name,
    north=False,
    south=True,
)

This demonstrator does not use the ERA5 climate reanalysis data as mentioned above since an private API key should be set up per user to access the CDS API. However, if it was set up, the ERA5 data can also be preprocessed by:

```python
pp = IceNetERA5PreProcessor(
    ["uas", "vas"],             # Absolute normalised variables
    ["tas", "zg500", "zg250"],  # Variables defined as deviations from an aggregated norm
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True
)
```

Next, we initialise the data processors using `init_source_data` which scans the data source directories to understand what data is available for processing based on the parameters. Since we named the processed data `"notebook_api_data"` above, it will create a data loader config file, `loader.notebook_api_data.json`, in the current directory.

In [ ]:
osi.init_source_data(
    lag_days=1,
)
osi.process()

meta.process()

At this point the preprocessed data is ready to convert or create a configuration for the network dataset.

### Dataset creation

Now, we can create a dataset configuration for training the network.This can include cached data for the network in the format of a TFRecordDataset compatible set of tfrecords. To achieve this we create the `IceNetDataLoader`, which can both generate `IceNetDataSet` configurations (which easily provide the necessary functionality for training and prediction) as well as individual data samples for direct usage.

In [ ]:
from icenet.data.loaders import IceNetDataLoaderFactory

implementation = "dask"
loader_config = "loader.notebook_api_data.json"
dataset_name = "api_dataset"
lag = 1

dl = IceNetDataLoaderFactory().create_data_loader(
    implementation,
    loader_config,
    dataset_name,
    lag,
    n_forecast_days=7,
    north=False,
    south=True,
    output_batch_size=4,
    generate_workers=2)

We can see the loader config contains information about the data sources included and also the different dates to use for the training, validation and test sets:

In [ ]:
dl._config

At this point we can either use `generate` or `write_dataset_config_only` to produce a ready-to-go `IceNetDataSet` configuration. Both of these will generate a dataset config, `dataset_config.api_dataset.json` (recall we set the dataset name as `api_dataset` above).

In [ ]:
dl.generate()

INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:35675'. Reason: worker-close
INFO:distributed.nanny:Closing Nanny gracefully at 'tcp://127.0.0.1:42976'. Reason: worker-close
INFO:distributed.scheduler:Remove client Client-worker-d2427ba6-e216-11ee-a878-ec2a7245f900
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:36340; closing.
INFO:distributed.scheduler:Remove client Client-worker-d2427ba6-e216-11ee-a878-ec2a7245f900
INFO:distributed.scheduler:Remove client Client-worker-dd607a9a-e216-11ee-a874-ec2a7245f900
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:36434; closing.
INFO:distributed.scheduler:Remove client Client-worker-dd607a9a-e216-11ee-a874-ec2a7245f900
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:36302; closing.
INFO:distributed.scheduler:Close client connection: Client-worker-d2427ba6-e216-11ee-a878-ec2a7245f900
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:39111', name: 1

To generate samples from this dataset, we can use the `.generate_sample()` method, which returns the inputs `x`, `y` and sample weights `sw`:

In [ ]:
x, y, sw = dl.generate_sample(pd.Timestamp("2020-03-08"))

In [ ]:
print(f"type(x): {type(x)}, x.shape: {x.shape}")
print(f"type(y): {type(y)}, y.shape: {y.shape}")
print(f"type(sw): {type(sw)}, sw.shape: {sw.shape}")

___
# Train

For single runs we programmatically can call the same method used by the CLI. `train_model` defines the training process from start to finish. The [`model-ensembler`](https://github.com/JimCircadian/model-ensembler) works outside the API, controlling multiple CLI submissions. Customising an ensemble can be achieved through looking at the configuration in [the pipeline repository](https://github.com/antarctica/IceNet-Pipeline). That said, if workflow system integration (e.g. Airflow) is desired, integrating via this method is the way to go.

In [ ]:
from icenet.data.dataset import IceNetDataSet

dataset_config = f"dataset_config.{dataset_name}.json"
dataset = IceNetDataSet(dataset_config, batch_size=4)
strategy = tf.distribute.get_strategy()

In [ ]:
dataset._config

You can obtain the data loader that was used to create the dataset config via the `.get_data_loader()` method:

In [ ]:
dataset.get_data_loader()

We can use `train_model` function to train.

In [ ]:
from icenet.model.train import train_model

run_name = "api_test_run"
seed = 42

trained_path, history = train_model(
    run_name="api_test_run",
    dataset=dataset,
    epochs=10,
    n_filters_factor=0.3,
    seed=seed,
    strategy=strategy,
    training_verbosity=2,
)

As can be seen the training workflow is very standard for deep learning networks, with `train_model` wrapping up the training process with a lot of customisation of extraneous functionality.

For a higher level of customisation programmatically, the training function can be split apart.

___
# Predict

In much the same manner as with `train_model`, the `predict_forecast` method acts as a convenient entry point workflow system integration, CLI entry as well as an overridable method upon which to base custom implementations. Using the method directly relies on loading from a prepared (but perhaps not cached) dataset.

Some parameters are fed to `predict_forecast` that ideally shouldn't need to be specified (like `seed` and `n_filters_factor`) and might seem contextually odd. They're used to locate the appropriate saved network. *This will be cleaned up in a future version*.

In [ ]:
from icenet.model.predict import predict_forecast

# Follows the naming convention used by the CLI version
output_dir = os.path.join(".", "results", "predict",
                          "custom_run_forecast",
                          "{}.{}".format(run_name, "42"))

predict_forecast(
    dataset_config=dataset_config,
    network_name=run_name,
    n_filters_factor=0.3,
    output_folder=output_dir,
    seed=seed,
    start_dates=[pd.to_datetime(el).date()
                 for el in pd.date_range("2020-03-08", "2020-03-09")],
    test_set=True,
)

___
# Visualisation

In [ ]:
!printf "2020-04-01\n2020-04-02" | tee predict_dates.csv

In [ ]:
!icenet_output -m -o ./results/predict custom_run_forecast api_dataset predict_dates.csv

In [ ]:
from icenet.plotting.video import xarray_to_video as xvid
from icenet.data.sic.mask import Masks
from IPython.display import HTML
import xarray as xr, pandas as pd, datetime as dt

# Load our output prediction file
ds = xr.open_dataset("results/predict/custom_run_forecast.nc")
land_mask = Masks(south=True, north=False).get_land_mask()
ds.info()

In [ ]:
# Get the forecast start date
forecast_date = ds.time.values[0]
print(forecast_date)

In [ ]:
fc = ds.sic_mean.isel(time=0).drop_vars("time").rename(dict(leadtime="time"))
fc['time'] = [pd.to_datetime(forecast_date) \
              + dt.timedelta(days=int(e)) for e in fc.time.values]

anim = xvid(fc, 15, figsize=4, mask=land_mask)
HTML(anim.to_jshtml())

## Summary
*Provide 3-5 bullet points summarising the main aspects of the dataset and tools covered in the notebook.*

* Sentence 1 e.g. `tool-name` to perform...
* Sentence 2 e.g. `tool-name` to perform...

## Additional information
**Dataset**: Type here details of dataset(s) version.

**Codebase**: Type here details of codebase version (only for notebooks categorised under modelling/preprocesing/post-processing themes).

**License**: The code in this notebook is licensed under the MIT License. The Environmental Data Science book is licensed under the Creative Commons by Attribution 4.0 license. See further details [here](https://github.com/alan-turing-institute/environmental-ds-book/blob/master/LICENSE.md).

**Contact**: If you have any suggestion or report an issue with this notebook, feel free to [create an issue](https://github.com/alan-turing-institute/environmental-ds-book/issues/new/choose) or send a direct message to [environmental.ds.book@gmail.com](mailto:environmental.ds.book@gmail.com).

In [ ]:
import icenet
icenet_version = icenet.__version__
print(f'IceNet version: {icenet_version}')

In [ ]:
from datetime import date
print(f'Last tested: {date.today()}')